In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#load libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import os
print(os.listdir("../input"))

In [ ]:
data = pd.read_csv("/kaggle/input/predicting-pulsar-starintermediate/pulsar_data_train.csv")
data


In [ ]:
data.columns = ['mean_profile', 'std_profile', 'kurtosis_profile', 'skewness_profile', 'mean_dmsnr',
               'std_dmsnr', 'kurtosis_dmsnr', 'skewness_dmsnr', 'target']

In [ ]:
data.dropna(inplace=True)
data.isna().sum()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data["target"].value_counts()

In [ ]:
data.groupby(by="target").agg(["min","mean","max"])

In [ ]:
data1 = data.groupby('target')[['mean_profile','std_profile','kurtosis_profile','skewness_profile', 'mean_dmsnr',
                      'std_dmsnr', 'kurtosis_dmsnr', 'skewness_dmsnr']].mean().reset_index()

data1= data1.transpose().reset_index()
data1.columns = ['features', 'Not Pulsar Star', 'Pulsar Star']

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize=(15,13))
plt.subplot(211)
vis2=sns.pointplot(data=data1.iloc[1:], x='features', y='Not Pulsar Star',color='red', label='Not Pulsar Star')
vis3=sns.pointplot(data=data1.iloc[1:], x='features', y='Pulsar Star', color='green', label='Pulsar Star')

plt.title('Mean values of features for every target class', fontsize=15)
plt.xlabel('Feature', fontsize=13)
plt.ylabel('Values', fontsize=13)
plt.xticks(rotation=30)

In [ ]:
pulsar = data[data['target'] ==1]
pulsar_count = pulsar["target"].value_counts()[1]
not_pulsar = data[data['target'] == 0]
not_pulsar_count = not_pulsar["target"].value_counts()[0]

In [ ]:
#pie plotting the stats between pulsars and not pulsars
plt.figure(figsize=(5,5))
plt.pie(data["target"].value_counts().values,labels=["not pulsar stars","pulsar stars"], autopct="%1.0f%%")
plt.title("Proportion of target variable in dataset")
plt.show()
print("There are " + str(pulsar_count) + " signals that belong to pulsar stars " 
      + "and " + str(not_pulsar_count) + " signals that aren't from pulsars.")

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(x=data['kurtosis_profile'], y=data['skewness_profile'], hue=data['target'])

In [ ]:
plt.figure(figsize=(25,25))
sns.lmplot(x="std_dmsnr", y="std_profile", hue="target", data=data,  markers=["x", "o"])


In [ ]:
y=data.target.values
x_data=data.drop(["target"], axis=1)
data.dropna(inplace=True) 
x=(x_data-np.min(x_data))/(np.max(x_data)-np.min(x_data) )
from sklearn.model_selection import train_test_split

x_train, x_test, y_train,y_test=train_test_split(x,y,test_size=0.3, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(x_train,y_train)
print(lr.score(x_test,y_test))
print(lr.score(x_train,y_train))

In [ ]:
from sklearn.linear_model import Ridge
ridge= Ridge().fit(x_train,y_train)
print(ridge.score(x_test,y_test))
print(ridge.score(x_train,y_train))

In [ ]:
ridge10=Ridge(alpha=10).fit(x_train,y_train)
print(ridge10.score(x_train,y_train))
print(ridge10.score(x_test,y_test))

In [ ]:
ridge01=Ridge(alpha=0.1).fit(x_train,y_train)
print(ridge01.score(x_train,y_train))
print(ridge01.score(x_test,y_test))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier(n_neighbors=7,weights="distance")

knn_model.fit(x_train,y_train)

y_head_knn = knn_model.predict(x_test)

knn_score = knn_model.score(x_test,y_test)

knn_score


In [ ]:
sns.set_style("whitegrid")
knn_score_list = []
for i in range(1,20):
    knn_2 = KNeighborsClassifier(n_neighbors = i)
    knn_2.fit(x_train, y_train)
    knn_score_list.append(knn_2.score(x_test, y_test))
 
plt.figure(figsize=(15,8))
plt.plot(range(1,20), knn_score_list)
plt.xticks(np.arange(1,20,1))
plt.xlabel("K values")
plt.ylabel("Skor")

knn_acc_2_max = max(knn_score_list)*100
print("Maksimum KNN skoru {:.2f}%".format(knn_acc_2_max))

In [ ]:
train_accuracy=[]
test_accuracy=[]
neighborhood_numbers=range(1,11)

In [ ]:
sns.set_style("whitegrid")
for n_neighborhood in neighborhood_numbers:
    snf=KNeighborsClassifier(n_neighbors=n_neighborhood)
    snf.fit(x_train,y_train)
    train_accuracy.append(snf.score(x_train,y_train))
    test_accuracy.append(snf.score(x_test,y_test))
    
plt.plot(neighborhood_numbers,train_accuracy,label='train accuracy')
plt.plot(neighborhood_numbers,test_accuracy,label='test accuracy')
plt.ylabel('accuracy')
plt.xlabel('n_neighborhood')

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree=DecisionTreeClassifier()
tree.fit(x_train,y_train)

In [ ]:
print(tree.score(x_train,y_train))
print(tree.score(x_test,y_test))

In [ ]:
tree=DecisionTreeClassifier(max_depth=4)
tree.fit(x_train,y_train)

In [ ]:
print(tree.score(x_train,y_train))
print(tree.score(x_test,y_test))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest=RandomForestClassifier(n_estimators=100).fit(x_train,y_train) 
print(forest.score(x_test,y_test))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbrt=GradientBoostingClassifier(learning_rate=0.01,random_state=0).fit(x_train,y_train)

In [ ]:
print(gbrt.score(x_train,y_train))
print(gbrt.score(x_test,y_test))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=150, random_state=1)
rf.fit(x_train,y_train)
print("Score for random forest {}".format(rf.score(x_test,y_test)))

In [ ]:
y_prediction=rf.predict(x_test)
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_prediction)
cm

In [ ]:
f, ax=plt.subplots(figsize=(10,10))
sns.heatmap(cm, annot=True, linewidths=0.5, linecolor="red", fmt=".0f", ax=ax)
plt.xlabel("y_prediction")
plt.ylabel("y_test")
plt.show()

In [ ]:
features = data.iloc[:,0:8]
plt.figure(figsize=(15,20))
j = 0
for i in features:
    plt.subplot(4,3,j+1)
    sns.violinplot(x=data["target"],y=data[i],palette=["red","blue"])
    plt.title(i)
    plt.axhline(data[i].mean(),linestyle = "dashed", label ="Ortalama değer = " + str(round(data[i].mean(), 2)))
    plt.legend(loc="best")
    j = j + 1

    

In [ ]:
plt.figure(figsize=(15,8))
plt.subplot(121)
sns.scatterplot(data=data[data['target']==1], x='kurtosis_profile', y='skewness_profile', label='Pulsar Yıldızı Değil')
sns.scatterplot(data=data[data['target']==0], x='kurtosis_profile', y='skewness_profile', label='Pulsar Star')
plt.title('Profile', fontsize=15)
plt.xlabel('Kurtosis profile', fontsize=13)
plt.ylabel('Skweness profile', fontsize=13)

plt.subplot(122)
sns.scatterplot(data=data[data['target']==0], x='kurtosis_dmsnr', y='skewness_dmsnr', label='Pulsaryıldızı Değil')
sns.scatterplot(data=data[data['target']==1], x='kurtosis_dmsnr', y='skewness_dmsnr', label='Pulsar Yıldız')
plt.title('DMSNR', fontsize=15)
plt.xlabel('Kurtosis DMSNR', fontsize=13)
plt.ylabel('Skweness DMSNR', fontsize=13)

In [ ]:
#plot correlation matrix
plt.figure(figsize=(15,15))
sns.heatmap(data.iloc[:,0:9].corr(), annot=True, fmt='.0%')

In [ ]:
sns.pairplot(data, hue="target", palette=["red","lime"])